In [1]:
import pandas as pd
import config
from pathlib import Path
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = config.GOOGLE_CREDENTIALS

In [2]:
# Creating Agent
llm = OpenAI(temperature=0, verbose=True, openai_api_key=config.OPENAI_API_KEY)

/Users/alejandromunoz/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [3]:
from langchain.sql_database import SQLDatabase
from sqlalchemy import create_engine

# Definir el proyecto y dataset
PROJECT_ID = "acoustic-env-452102-r3"
DATASET_ID = "olist"

# Crear la URI de conexión
BIGQUERY_URI = f"bigquery://{PROJECT_ID}/{DATASET_ID}"

# Crear el motor SQLAlchemy para BigQuery
engine = create_engine(BIGQUERY_URI)


In [4]:
# Conectar la base de datos a LangChain
db = SQLDatabase(engine)

/Users/alejandromunoz/Library/Python/3.9/lib/python/site-packages/google/cloud/bigquery/client.py:587: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


In [5]:
# Probar la conexión ejecutando una consulta en la tabla "olist.ecommerce_data"
query = "SELECT * FROM olist.ecommerce_data LIMIT 5"
result = db.run(query)

In [6]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [7]:
agent_executor.run(
    "Cual es el producto más caro"
)

/Users/alejandromunoz/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ecommerce_data, olist_customers, olist_geolocation, olist_order_items, olist_order_payments I should query the schema of the olist_order_items table to see what columns are relevant to the question.
Action: sql_db_schema
Action Input: olist_order_items
CREATE TABLE `olist_order_items` (
	`order_id` STRING, 
	`order_item_id` STRING, 
	`product_id` STRING, 
	`seller_id` STRING, 
	`shipping_limit_date` STRING, 
	`price` STRING, 
	`freight_value` STRING
)

/*
3 rows from olist_order_items table:
order_id	order_item_id	product_id	seller_id	shipping_limit_date	price	freight_value
3ee6513ae7ea23bdfab5b9ab60bffcb5	1	8a3254bee785a526d548a81a9bc3c9be	96804ea39d96eb908e7c3afdb671bb9e	2018-05-04 03:55:26	0.85	18.23
6e864b3f0ec71031117ad4cf46b7f2a1	1	8a3254bee785a526d548a81a9bc3c9be	96804ea39d96eb908e7c3afdb671bb9e	2018-05-02 20:30:34	0.85	18.23
c5bdd8ef3c0ec420232e668302179113	2	8a3254bee785a526d548a81a9bc3c9be	9

'The product with the highest price is 8a3254bee785a526d548a81a9bc3c9be with a price of 0.85.'